In [79]:
import pandas as pd
from datetime import datetime
import numpy as np

import seaborn as sns

In [3]:
df_customers = pd.read_csv("kulichevsky_customers.csv")
df_orders = pd.read_csv("kulichevsky_orders.csv")

In [17]:
df_orders.head()

,id,order_date,ship_mode,customer_id,sales
0,100006,2014-09-07,Standard,DK-13375,377.970
1,100090,2014-07-08,Standard,EB-13705,699.192
2,100293,2014-03-14,Standard,NF-18475,91.056
3,100328,2014-01-28,Standard,JC-15340,3.928
4,100363,2014-04-08,Standard,JM-15655,21.376


In [18]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 5 columns):
id             5009 non-null int64
order_date     5009 non-null object
ship_mode      5009 non-null object
customer_id    5009 non-null object
sales          5009 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 195.7+ KB


In [20]:
df_orders["order_date"] = pd.to_datetime(df_orders["order_date"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 5 columns):
id             5009 non-null int64
order_date     5009 non-null datetime64[ns]
ship_mode      5009 non-null object
customer_id    5009 non-null object
sales          5009 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 195.7+ KB


## 1. Сколько заказов, отправлено первым классом за последние 5 лет?

787

In [30]:
df_orders[(df_orders["ship_mode"] == "First") & 
          (df_orders["order_date"].dt.year >= 2014)]["id"].count()

787

## 2. Сколько в базе клиентов из Калифорнии?
161

In [55]:
df_customers[df_customers['state'] == "California"]["id"].count()


161

## 3. Сколько заказов они сделали?

1006

In [65]:
new_df = df_orders.merge(df_customers, 
                         how = "left", 
                         left_on = "customer_id",
                         right_on = "id")

new_df[new_df['state'] == "California"]["id_x"].count()

1006

## 4. Постройте сводную таблицу средних чеков по всем штатам за каждый год.

In [66]:
new_df.head()

,id_x,order_date,ship_mode,customer_id,sales,id_y,name,segment,state,city
0,100006,2014-09-07,Standard,DK-13375,377.970,DK-13375,Dennis Kane,Consumer,Ohio,Marion
1,100090,2014-07-08,Standard,EB-13705,699.192,EB-13705,Ed Braxton,Corporate,Ohio,Akron
2,100293,2014-03-14,Standard,NF-18475,91.056,NF-18475,Neil Franzosisch,Home Office,Illinois,Rockford
3,100328,2014-01-28,Standard,JC-15340,3.928,JC-15340,Jasper Cacioppo,Consumer,California,Los Angeles
4,100363,2014-04-08,Standard,JM-15655,21.376,JM-15655,Jim Mitchum,Corporate,New Jersey,Bridgeton


In [133]:
cm = sns.light_palette("green", as_cmap=True)

df_pivot = new_df.pivot_table(values= ["sales"], 
                   index= new_df["order_date"].dt.year, 
                   columns = new_df["state"], 
                   aggfunc = "mean")
                   

df_pivot.style.format("{:.0f}")\
        .highlight_max(axis=1)

In [137]:
max_mean_sales_by_year = df_pivot.apply(lambda x: x.idxmax(axis = 1)[1], axis = 1)

max_mean_sales_by_year.head()

order_date
2014    North Carolina
2015       Mississippi
2016     Massachusetts
2017            Kansas
dtype: object